In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import dash 
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

In [2]:
path = '/users/anirudh/downloads/spacex_launch_dash.csv'
spacex_df = pd.read_csv(path, index_col = None)
spacex_df.drop('Unnamed: 0', axis = 1, inplace = True)
spacex_df.head()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [3]:
test = spacex_df[spacex_df['Launch Site'] == 'KSC LC-39A']
test['class'].value_counts()

class
1    10
0     3
Name: count, dtype: int64

In [7]:
app6 =dash.Dash(__name__)

app6.layout = html.Div(children = [html.H1("SpaceX Launch Records Dashboard", style ={'textAlign':'center', 'color':'black', 'font-size':25}),
                                   dcc.Dropdown(id ='site-dropdown',
                                                options = [
                                                    {'label':'All Sites','value':'ALL'},
                                                    {'label':'Site 1: CCAFS LC-40','value':'CCAFS LC-40'},
                                                    {'label':'Site 2: CCAFS SLC-40','value':'CCAFS SLC-40'},
                                                    {'label':'Site 3: VAFB SLC-4E','value':'VAFB SLC-4E'},
                                                    {'label':'Site 4: KSC LC-39A','value':'KSC LC-39A'}
                                                ],
                                                placeholder = 'Select A Launch Site',
                                                value = 'ALL',
                                                searchable = True,
                                                style = {'width':'80%', 'padding':'3px', 'font-size':'20px', 'textAlign':'center'}
                                               ),
                                   html.Div([
                                       dcc.Graph(id='success-pie-chart', className='chart-grid', style={'display':'flex'}
                                                 )
                                             ]),
                                   dcc.RangeSlider(id='payload-slider', min=0, max=10000, step =1000,
                                                   value = [0, 10000]
                                                  ),
                                   html.Div(id ='output-container-slider'),
                                   html.Div([
                                       dcc.Graph(id='success-payload-scatter-chart', className='chart-grid', style={'display':'flex'}
                                                )
                                             ])
                                  
                                  ])
@app6.callback(
    Output(component_id = 'output-container-slider', component_property = 'children'),
    Input(component_id = 'payload-slider', component_property='value')
)

def update_output(value):
    return ("You have selected {}".format(value))

@app6.callback(
    Output(component_id = 'success-pie-chart', component_property = 'figure'),
    Input(component_id = 'site-dropdown', component_property = 'value')
)

def get_pie_chart(site_value):
    if site_value == 'ALL':
        data = spacex_df
        figure = px.pie(data, values = 'class', names = 'Launch Site', color = 'Launch Site',
                        color_discrete_map={'CCAFS LC-40':'seablue','CCAFS SLC-40':'cyan',
                                            'VAFB SLC-4E':'royalblue', 'KSC LC-39A':'darkblue'},
        title = 'Launch Site Success Comparison')
        return figure
    else:
        data = spacex_df[spacex_df['Launch Site'] == str(site_value)]
        list1 = []
        for value in data['class'].value_counts():
            list1.append(value)
        
        fig = px.pie(values = list1,names = ['Success', 'Fail'], color_discrete_sequence=px.colors.sequential.RdBu,
                     title = 'Success Rate Pie Chart for Launch Site {}'.format(site_value))
        return fig

@app6.callback(
    Output(component_id = 'success-payload-scatter-chart', component_property = 'figure'),
    [Input(component_id = 'site-dropdown', component_property = 'value'),
    Input(component_id = 'payload-slider', component_property = 'value')]
)

def get_payload_scatter(site_value,slider):
    if site_value == 'ALL':
        data = spacex_df[(spacex_df['Payload Mass (kg)'] > slider[0]) & (spacex_df['Payload Mass (kg)'] < slider[1])]
        
        figure = px.scatter(data, x = 'Payload Mass (kg)' , y= 'class', color = 'Booster Version Category',
                            title= 'Correlation Between Payload Mass and Success for All Sites',
                           color_discrete_map={'FT':'#e377c2','v1.1':'#d62728',
                                            'B5':'royalblue', 'B4':'#2ca02c', 'v1.0':'black'},
                           width = 940, height = 350)
        figure.update_layout(yaxis = dict(tickvals = [0,1], ticktext = ['O:Failure', '1:Success']))
        
        return figure
    else:
        data = spacex_df[spacex_df['Launch Site'] == str(site_value)]
        data = data[(data['Payload Mass (kg)'] > slider[0]) & (data['Payload Mass (kg)'] < slider[1])]
        
        figure = px.scatter(data, x = 'Payload Mass (kg)' , y= 'class', color = 'Booster Version Category',
                            title= 'Correlation Between Payload Mass and Success for Site {}'.format(site_value),
                           color_discrete_map={'FT':'#e377c2','v1.1':'#d62728',
                                            'B5':'royalblue', 'B4':'#2ca02c', 'v1.0':'black'},
                           width = 940, height = 350)
        figure.update_layout(yaxis = dict(tickvals = [0,1], ticktext = ['O:Failure', '1:Success']))
        return figure 
    

if __name__ == '__main__':
    app6.run_server(port = 8080)   